In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# Курсовая

## Вариант 19  
Решить краевую задачу для обыкновенного дифференциального уравнения второго порядка  
$$
\begin{equation*}
\begin{cases}
xy''-(2x+1)y'+(x+1)y=0\\
y'(1)=3e\\
py(2)+qy'(2)=0
\end{cases}
\end{equation*}
$$  
где *p* и *q* удовлетворяют системе уравнений
$$
\begin{equation*}
\begin{cases}
sin(p+0.5) + q = 0\\
cos(q-0.5) - 1.1p = 3
\end{cases}
\end{equation*}
$$
----------------

### Решение задачи  
Решение задачи будет заключаться в решениях СНАУ *методом Ньютона* и краевой задачи для ОДУ второго порядка *конечно-разностным методом*.  
_____________

#### Метод Ньютона  
*Ответ по desmos.com: (-1.925, 0.989)*  
$$
F(x) = 
\begin{pmatrix}
sin(p+0.5)+q \\
cos(q-0.5)-1.1p-3
\end{pmatrix}
$$
$$
W(x) = 
\begin{pmatrix}
cos(p+0.5) && 1 \\
-1.1 && -sin(q-0.5) \\
\end{pmatrix}
$$

In [ ]:
#Система нелинейных алгебраических уравнений
def f(p, q): return np.array(
    [np.sin(p + .5) + q, np.cos(q - .5) - 1.1*p - 3])

def w(p, q): return np.array(
    [[np.cos(p + .5), 1], [-1.1, -np.sin(q - .5)]]) #Матрица Якоби

##### График

In [57]:
t = np.linspace(-10, 10, 50)
p, q = np.array([-np.sin(i+.5) for i in t]), np.array([(np.cos(i-.5)-3)/1.1 for i in t])
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=t, y=p, mode='lines+markers', name='q + sin(p+0.5) = 0')
)
fig.add_trace(
    go.Scatter(x=q, y=t, mode='lines+markers', name='cos(q - 0.5) - 1.1 p = 3')
)

fig.update_layout(
    legend_orientation='h',
    legend=dict(x=.5, xanchor="center"),
    title='Графики уравнений',
    title_x = 0.5,
    xaxis_title='<b>p</b>',
    yaxis_title='<b>q</b>',
    template='plotly_white',
    width=1000, height=1000
)

fig.update_xaxes(
    title_font=dict(
        size=26,
        family='Times New Roman'
    )
)
fig.update_yaxes(
    title_font=dict(
        size=26,
        family='Times New Roman'
    )
)

fig.show()

##### Решение подзадачи

In [58]:
from main import method_newton

X = [-2., 1.] #Примерное приближенное решения

iters = method_newton(X, f, w)

In [59]:
pd.DataFrame(data={
    'Итерация': iters.keys(),
    'p': [i[0] for i in iters.values()],
    'q': [i[1] for i in iters.values()]
}
)

,Итерация,p,q
0,0,-2.000000,1.000000
1,1,-1.926100,0.992268
2,2,-1.924881,0.989374
3,3,-1.924884,0.989374


_________________________________
#### Конечно-разностный метод  
ОДУ должно соответствовать следующим краевым условиям:
$$
\begin{equation*}
\begin{cases}
    y'' + p(x)y' + q(x)y=f(x)\\
    y(a)=y_a\\ 
    y(b)=y_b
\end{cases}
\end{equation*}
$$
$$
p(x) = -\frac{2x+1}{x}\\
\;\\
q(x) = \frac{x+1}{x}\\
\;\\
f(x) = 0
$$  
Для граничных условий второго рода аппроксимация производных проводится с помощью односторонних разностей первого порядка. Для нашего случая получается:  
$$
\begin{equation*}
\begin{cases}
-y_0+y_1=3*e*h\\
-q*y_{n-1} + y_n*(p*h+q)=0
\end{cases}
\end{equation*}
$$

##### Функция для построения графиков

In [204]:
def update_graphs(fig, x_arr:list, y_arr:list, h:float):
    my_color = ('rgba('+str(np.random.randint(0, high = 256))+','+
                str(np.random.randint(0, high = 256))+','+
                str(np.random.randint(0, high = 256)))

    fig.add_trace(
        go.Scatter(x=x_arr, y=y_arr, mode='lines', name=f'h={h}', line=dict(dash='longdashdot'))
    )
    return fig

##### Начальные условия

In [ ]:
from main import fin_diff
fig = go.Figure()

res_p, res_q = iters[3] # p, q из решения СНАУ методом Ньютона

a, b, h = 1, 2, [1, 0.5, 0.2, 0.1, 0.01, 0.001] # Границы(левая и правая), шаг

p_x = lambda x: -(2*x+1)/x # коэффициент перед y'
q_x = lambda x: (x+1)/x # коэффициент перед y
f_x = lambda x: 0 # свободный коэффициент

##### Решение ОДУ конечно-разностным методом

In [201]:
for h_i in h:
    x_arr, y_arr = fin_diff(a, b, h_i, p_x, q_x, f_x, [-1, 1, 3*np.exp(1)*h_i], [-res_q, (res_p*h_i+res_q), 0])
    fig = (update_graphs(fig, x_arr, y_arr, h_i))

##### Построение графика

In [202]:
fig.update_layout(
        legend_orientation='h',
        legend=dict(x=.5, xanchor='auto'),
        title='График решения краевой задачи для ОДУ второго порядка Конечно-разностным методом',
        title_x = 0.5,
        xaxis_title='<b>x</b>',
        yaxis_title='<b>y(x)</b>',
        template='plotly_white',
        width=1000, height=1000
    )

fig.update_xaxes(
    title_font=dict(
        size=26,
        family='Times New Roman'
    )
)
fig.update_yaxes(
    title_font=dict(
        size=26,
        family='Times New Roman'
    )
)

fig.show()